In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.utils import plot_model

import plotly.express as px # for interactive plots

#Load the Electrical Load Data

In [3]:
DATA_DIR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Load"
hist_window=18;forecast_window=18
#hist_window=int(12*60/5)

In [4]:
data = []
for y in range(2017,2023):
  df = pd.read_csv(f"{DATA_DIR}/Load_{y}.csv")
  data.append(df)
data = pd.concat(data)

NYC = data[['Time Stamp','N.Y.C.']];
NYC['Time Stamp']=pd.to_datetime(NYC['Time Stamp'])

NYCA = data[['Time Stamp','NYCA']];
NYCA['Time Stamp']=pd.to_datetime(NYCA['Time Stamp']) 

NYC = NYC.set_index('Time Stamp')
NYC = NYC.asfreq('5min')
NYC = NYC.reset_index()


print(NYC.head(15))#;print(len(NYC.index),NYC.mean(),NYC.std())
print(NYCA.head(15))

            Time Stamp  N.Y.C.
0  2017-01-01 00:00:00  5116.9
1  2017-01-01 00:05:00  5102.1
2  2017-01-01 00:10:00  5074.9
3  2017-01-01 00:15:00  5089.6
4  2017-01-01 00:20:00  5061.3
5  2017-01-01 00:25:00  5042.6
6  2017-01-01 00:30:00  5011.6
7  2017-01-01 00:35:00  4999.3
8  2017-01-01 00:40:00  4982.0
9  2017-01-01 00:45:00  4972.1
10 2017-01-01 00:50:00  4957.4
11 2017-01-01 00:55:00  4965.4
12 2017-01-01 01:00:00  4956.1
13 2017-01-01 01:05:00  4913.0
14 2017-01-01 01:10:00  4888.8
            Time Stamp     NYCA
0  2017-01-01 00:00:00  16134.7
1  2017-01-01 00:05:00  16048.7
2  2017-01-01 00:10:00  15902.4
3  2017-01-01 00:15:00  15911.3
4  2017-01-01 00:20:00  15880.3
5  2017-01-01 00:25:00  15806.3
6  2017-01-01 00:30:00  15725.1
7  2017-01-01 00:35:00  15653.8
8  2017-01-01 00:40:00  15605.4
9  2017-01-01 00:45:00  15557.5
10 2017-01-01 00:50:00  15471.6
11 2017-01-01 00:55:00  15461.4
12 2017-01-01 01:00:00  15335.6
13 2017-01-01 01:05:00  15288.5
14 2017-01-01 01:10:00  

<ipython-input-4-9ceaafef6e74>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NYC['Time Stamp']=pd.to_datetime(NYC['Time Stamp'])
<ipython-input-4-9ceaafef6e74>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NYCA['Time Stamp']=pd.to_datetime(NYCA['Time Stamp'])


In [21]:
data

,Time Stamp,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,NORTH,WEST,NYCA
0,2017-01-01 00:00:00,1211.6000,1671.9000,620.2000,978.6000,1012.3000,2161.0000,867.2000,320.6000,5116.9000,539.2000,1635.2000,16134.7000
1,2017-01-01 00:05:00,1217.2000,1667.5000,610.4000,967.8000,1008.4000,2139.6000,848.1000,309.0000,5102.1000,542.5000,1636.1000,16048.7000
2,2017-01-01 00:10:00,1210.1000,1613.6000,622.1000,960.6000,947.0000,2131.8000,857.2000,326.9000,5074.9000,527.8000,1630.4000,15902.4000
3,2017-01-01 00:15:00,1205.2000,1629.4000,594.4000,953.4000,984.6000,2115.6000,866.2000,305.8000,5089.6000,531.5000,1635.6000,15911.3000
4,2017-01-01 00:20:00,1196.2000,1639.3000,599.2000,955.5000,991.1000,2110.6000,848.5000,305.3000,5061.3000,538.2000,1635.1000,15880.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,2022-12-31 23:35:00,1173.4756,1455.1769,561.4635,938.2208,1000.8982,1920.7468,697.0698,238.4149,4749.6870,661.1154,1472.5676,14868.8365
105116,2022-12-31 23:40:00,1164.8765,1474.7970,563.8666,930.3062,946.4758,1906.6010,696.8730,236.1247,4738.3370,657.3319,1463.1033,14778.6930
105117,2022-12-31 23:45:00,1163.8800,1476.9541,561.3690,929.1260,952.0700,1891.1267,698.8709,229.8545,4731.2676,658.4504,1458.7169,14751.6861
105118,2022-12-31 23:50:00,1154.3290,1421.4199,561.5217,929.3459,928.7479,1879.3613,703.1354,234.4483,4721.4570,653.5872,1435.5454,14622.8990


# New Solar Data


In [6]:
# path to solar data in drive
SOLAR_DATA_DIR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Solar_Irradiance"

# iterates over each solar datasets and reads it in

sol_df = pd.read_csv(f"{SOLAR_DATA_DIR}/NYC_Solcast_PT5M.csv")


In [27]:
sol_df.head()

,PeriodEnd,PeriodStart,Period,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,...,GtiFixedTilt,GtiTracking,PrecipitableWater,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily
0,2016-12-31T01:00:00Z,2016-12-31T00:55:00Z,PT5M,1.4,90,0.0,-5.8,0,0,0,...,0,0,5.9,58.5,0.0,1011.2,277,5.7,127,0.12
1,2016-12-31T01:05:00Z,2016-12-31T01:00:00Z,PT5M,1.4,89,0.0,-5.8,0,0,0,...,0,0,5.8,58.6,0.0,1011.3,278,5.7,128,0.12
2,2016-12-31T01:10:00Z,2016-12-31T01:05:00Z,PT5M,1.3,88,0.0,-5.9,0,0,0,...,0,0,5.8,58.6,0.0,1011.4,278,5.7,129,0.12
3,2016-12-31T01:15:00Z,2016-12-31T01:10:00Z,PT5M,1.3,87,0.0,-5.9,0,0,0,...,0,0,5.8,58.6,0.0,1011.4,278,5.8,130,0.12
4,2016-12-31T01:20:00Z,2016-12-31T01:15:00Z,PT5M,1.3,87,0.0,-6.0,0,0,0,...,0,0,5.8,58.7,0.0,1011.5,278,5.8,131,0.12


In [28]:
sol_df.shape

(645396, 21)

In [8]:
sol_df['datetime'] = pd.to_datetime(sol_df["PeriodEnd"]).dt.tz_convert('US/Eastern')
sol_df = sol_df.drop_duplicates(['datetime'])
sol_df['year'] = sol_df['datetime'].apply(lambda x: x.year)
sol_df['month'] = sol_df['datetime'].apply(lambda x: x.month)
sol_df['day'] = sol_df['datetime'].apply(lambda x: x.day)
sol_df['hour'] = sol_df['datetime'].apply(lambda x: x.hour)
sol_df['minute'] = sol_df['datetime'].apply(lambda x: x.minute)
sol_df['dow'] = sol_df['datetime'].apply(lambda x: x.dayofweek)
sol_df = sol_df.set_index('datetime')

data['datetime'] = pd.to_datetime(data['Time Stamp'])
# data = data.set_index('Time Stamp')
# data = data.asfreq('5min')
# data = data.reset_index()
data['year'] = data['datetime'].apply(lambda x: x.year)
data['month'] = data['datetime'].apply(lambda x: x.month)
data['day'] = data['datetime'].apply(lambda x: x.day)
data['hour'] = data['datetime'].apply(lambda x: x.hour)
data['minute'] = data['datetime'].apply(lambda x: x.minute)
data['dow'] = data['datetime'].apply(lambda x: x.dayofweek)

In [68]:
sol_df = sol_df.drop_duplicates()


In [38]:
# sol_df.index = sol_df['datetime']

sol_df = sol_df.asfreq('5T')

In [43]:
pd.to_datetime(sol_df["PeriodEnd"])

datetime
2016-12-30 20:00:00-05:00   2016-12-31 01:00:00+00:00
2016-12-30 20:05:00-05:00   2016-12-31 01:05:00+00:00
2016-12-30 20:10:00-05:00   2016-12-31 01:10:00+00:00
2016-12-30 20:15:00-05:00   2016-12-31 01:15:00+00:00
2016-12-30 20:20:00-05:00   2016-12-31 01:20:00+00:00
                                       ...           
2023-02-18 18:35:00-05:00   2023-02-18 23:35:00+00:00
2023-02-18 18:40:00-05:00   2023-02-18 23:40:00+00:00
2023-02-18 18:45:00-05:00   2023-02-18 23:45:00+00:00
2023-02-18 18:50:00-05:00   2023-02-18 23:50:00+00:00
2023-02-18 18:55:00-05:00   2023-02-18 23:55:00+00:00
Freq: 5T, Name: PeriodEnd, Length: 645396, dtype: datetime64[ns, UTC]

In [36]:
load_solar_df2

,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,NORTH,...,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,year,month,day,hour,minute,dow
Time Stamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,1211.6000,1671.9000,620.2000,978.6000,1012.3000,2161.0000,867.2000,320.6000,5116.9000,539.2000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01 00:05:00,1217.2000,1667.5000,610.4000,967.8000,1008.4000,2139.6000,848.1000,309.0000,5102.1000,542.5000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01 00:10:00,1210.1000,1613.6000,622.1000,960.6000,947.0000,2131.8000,857.2000,326.9000,5074.9000,527.8000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01 00:15:00,1205.2000,1629.4000,594.4000,953.4000,984.6000,2115.6000,866.2000,305.8000,5089.6000,531.5000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01 00:20:00,1196.2000,1639.3000,599.2000,955.5000,991.1000,2110.6000,848.5000,305.3000,5061.3000,538.2000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 23:35:00,1173.4756,1455.1769,561.4635,938.2208,1000.8982,1920.7468,697.0698,238.4149,4749.6870,661.1154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:40:00,1164.8765,1474.7970,563.8666,930.3062,946.4758,1906.6010,696.8730,236.1247,4738.3370,657.3319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:45:00,1163.8800,1476.9541,561.3690,929.1260,952.0700,1891.1267,698.8709,229.8545,4731.2676,658.4504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
load_solar_df = data.merge(sol_df, how='left', left_on=['year','month',	'day',	'hour',	'minute'], right_on=['year','month',	'day',	'hour',	'minute'])

In [11]:
load_solar_df = load_solar_df.drop_duplicates(['Time Stamp'])
load_solar_df

,Time Stamp,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,...,GtiTracking,PrecipitableWater,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,dow_y
0,2017-01-01 00:00:00,1211.6000,1671.9000,620.2000,978.6000,1012.3000,2161.0000,867.2000,320.6000,5116.9000,...,0.0,14.5,43.5,0.0,1011.9,226.0,7.0,162.0,0.12,6.0
1,2017-01-01 00:05:00,1217.2000,1667.5000,610.4000,967.8000,1008.4000,2139.6000,848.1000,309.0000,5102.1000,...,0.0,14.5,43.5,0.0,1011.9,227.0,7.0,162.0,0.13,6.0
2,2017-01-01 00:10:00,1210.1000,1613.6000,622.1000,960.6000,947.0000,2131.8000,857.2000,326.9000,5074.9000,...,0.0,14.5,43.5,0.0,1011.9,227.0,7.0,162.0,0.13,6.0
3,2017-01-01 00:15:00,1205.2000,1629.4000,594.4000,953.4000,984.6000,2115.6000,866.2000,305.8000,5089.6000,...,0.0,14.5,43.5,0.0,1011.9,228.0,7.0,162.0,0.13,6.0
4,2017-01-01 00:20:00,1196.2000,1639.3000,599.2000,955.5000,991.1000,2110.6000,848.5000,305.3000,5061.3000,...,0.0,14.5,43.5,0.0,1011.9,228.0,7.0,162.0,0.13,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631075,2022-12-31 23:35:00,1173.4756,1455.1769,561.4635,938.2208,1000.8982,1920.7468,697.0698,238.4149,4749.6870,...,0.0,27.5,99.2,0.0,1014.8,252.0,2.5,162.0,0.10,5.0
631076,2022-12-31 23:40:00,1164.8765,1474.7970,563.8666,930.3062,946.4758,1906.6010,696.8730,236.1247,4738.3370,...,0.0,27.2,99.2,0.0,1014.7,251.0,2.4,162.0,0.10,5.0
631077,2022-12-31 23:45:00,1163.8800,1476.9541,561.3690,929.1260,952.0700,1891.1267,698.8709,229.8545,4731.2676,...,0.0,26.9,99.2,0.0,1014.6,250.0,2.4,162.0,0.10,5.0
631078,2022-12-31 23:50:00,1154.3290,1421.4199,561.5217,929.3459,928.7479,1879.3613,703.1354,234.4483,4721.4570,...,0.0,26.6,99.2,0.0,1014.5,249.0,2.3,162.0,0.10,5.0


In [ ]:
sol_df.shape

(645396, 28)

In [ ]:
load_solar_df.shape

(645396, 43)

In [ ]:
load_solar_df[load_solar_df['NYCA'].isna()]

,Time Stamp,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,...,PrecipitableWater,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,datetime_y,dow_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.9,58.5,0.0,1011.2,277,5.7,127,0.12,2016-12-30 20:00:00-05:00,4
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.6,0.0,1011.3,278,5.7,128,0.12,2016-12-30 20:05:00-05:00,4
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.6,0.0,1011.4,278,5.7,129,0.12,2016-12-30 20:10:00-05:00,4
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.6,0.0,1011.4,278,5.8,130,0.12,2016-12-30 20:15:00-05:00,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.7,0.0,1011.5,278,5.8,131,0.12,2016-12-30 20:20:00-05:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,32.7,0.0,1035.0,249,4.5,102,0.12,2023-02-18 18:35:00-05:00,5
645392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,32.9,0.0,1035.0,248,4.5,103,0.12,2023-02-18 18:40:00-05:00,5
645393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,33.2,0.0,1035.1,247,4.5,104,0.12,2023-02-18 18:45:00-05:00,5
645394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,33.4,0.0,1035.1,246,4.5,105,0.12,2023-02-18 18:50:00-05:00,5


In [ ]:
fig = px.scatter(load_solar_df, x='AirTemp', y='NYCA', title='Interpolated Load vs Temperature')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
load_solar_df.index = load_solar_df['datetime_x']

In [ ]:
load_solar_df[load_solar_df.index.duplicated()]

,Time Stamp,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,...,PrecipitableWater,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,datetime_y,dow_y
datetime_x,,,,,,,,,,,,,,,,,,,,,
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.6,0.0,1011.3,278,5.7,128,0.12,2016-12-30 20:05:00-05:00,4
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.6,0.0,1011.4,278,5.7,129,0.12,2016-12-30 20:10:00-05:00,4
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.6,0.0,1011.4,278,5.8,130,0.12,2016-12-30 20:15:00-05:00,4
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.7,0.0,1011.5,278,5.8,131,0.12,2016-12-30 20:20:00-05:00,4
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8,58.7,0.0,1011.6,278,5.8,132,0.12,2016-12-30 20:25:00-05:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,32.7,0.0,1035.0,249,4.5,102,0.12,2023-02-18 18:35:00-05:00,5
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,32.9,0.0,1035.0,248,4.5,103,0.12,2023-02-18 18:40:00-05:00,5
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,33.2,0.0,1035.1,247,4.5,104,0.12,2023-02-18 18:45:00-05:00,5


In [ ]:
load_solar_df[load_solar_df.index.duplicated()]['Time Stamp'].values

array([nan, nan, nan, ..., nan, nan, nan], dtype=object)

In [ ]:
load_solar_df = load_solar_df.asfreq('5T')

ValueError: ignored

In [ ]:
load_solar_df.head()

In [ ]:
sol_df.describe()

# Old Solar Data

In [ ]:
# path to solar data in drive
SOLAR_DATA_DIR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Solar_Irradiance"

# iterates over each solar datasets and reads it in
solar_data = []
for y in range(2017,2021):
  df = pd.read_csv(f"{SOLAR_DATA_DIR}/solar_{y}.csv", skiprows=[0, 1])
  solar_data.append(df)



In [ ]:
all_solar = solar_data[0]
for df in solar_data[1:]:
  all_solar = all_solar.append(df, ignore_index=True)
all_solar["interpolated"] = False

# Merge and Interpolation

In [ ]:
# data['datetime'] = pd.to_datetime(data['Time Stamp'])
# data['year'] = data['datetime'].apply(lambda x: x.year)
# data['month'] = data['datetime'].apply(lambda x: x.month)
# data['day'] = data['datetime'].apply(lambda x: x.day)
# data['hour'] = data['datetime'].apply(lambda x: x.hour)
# data['minute'] = data['datetime'].apply(lambda x: x.minute)
# data['dow'] = data['datetime'].apply(lambda x: x.dayofweek)

# load_solar_df = data.merge(all_solar, how='left', left_on=['year','month',	'day',	'hour',	'minute'], right_on=['Year','Month',	'Day',	'Hour',	'Minute'])

# solar_cols = ['Temperature', 'Clearsky DHI',
#        'Clearsky DNI', 'Clearsky GHI', 'Dew Point', 'DHI', 'DNI',
#        'GHI', 'Relative Humidity', 'Solar Zenith Angle',
#        'Surface Albedo', 'Pressure', 'Precipitable Water', 'Wind Direction',
#        'Wind Speed', 'Global Horizontal UV Irradiance (280-400nm)',
#        'Global Horizontal UV Irradiance (295-385nm)']
# load_solar_df['interpolated'] = load_solar_df['interpolated'].fillna(value=True)
# load_solar_df.index = load_solar_df['datetime']

# load_solar_df_inter = load_solar_df.asfreq('5T')
# load_solar_df_inter[solar_cols] = load_solar_df_inter[solar_cols].interpolate(method='linear', order=1, limit=10, limit_direction='both')
# #load_solar_df_inter.to_csv(f"{SOLAR_DATA_DIR}/solar_full_5m.csv")


## Interpolate Verification

## Scatter Cloud

In [ ]:
fig = px.scatter(load_solar_df.sample(frac=1/12), x='AirTemp', y='NYCA', title='Interpolated Load vs Temperature')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter[load_solar_df_inter['interpolated']==False], x='Temperature', y='NYCA', title='Load vs Temperature')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter[load_solar_df_inter['interpolated']==True].sample(frac=1/12), x='Global Horizontal UV Irradiance (280-400nm)', y='NYCA', title='Load vs Interpolated Global Horizontal UV Irradiance (280-400nm)')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter[load_solar_df_inter['interpolated']==False], x='Global Horizontal UV Irradiance (280-400nm)', y='NYCA', title='Load vs Global Horizontal UV Irradiance (280-400nm)')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter[load_solar_df_inter['interpolated']==True].sample(frac=1/12), x='Global Horizontal UV Irradiance (280-400nm)', y='Temperature', title='Interpolated Temperature vs Global Horizontal UV Irradiance (280-400nm)')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter[load_solar_df_inter['interpolated']==False], x='Global Horizontal UV Irradiance (280-400nm)', y='Temperature', title='Load vs Global Horizontal UV Irradiance (280-400nm)')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()

In [ ]:
# create 3d scatter plot
fig = px.scatter_3d(load_solar_df_inter[load_solar_df_inter['interpolated']==True].sample(frac=1/12), x='Temperature', y='Global Horizontal UV Irradiance (295-385nm)', z='NYCA',
              color='Solar Zenith Angle')

# adjust marker size to see all data points
fig.update_traces(marker_size = 2)

# adjust layout
fig.update_layout(
    autosize=False,
    width=1200,
    height=800
)
fig.show()

## Time Verification

In [ ]:
fig = px.scatter(load_solar_df_inter.head(10000), x="datetime", y='Global Horizontal UV Irradiance (295-385nm)', color='interpolated', title='Global Horizontal UV Irradiance over Time')
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter.head(100000), x="datetime", y='Global Horizontal UV Irradiance (295-385nm)', color='interpolated', title='Global Horizontal UV Irradiance over Time')
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter.head(10000), x="datetime", y='Temperature', color='interpolated', title='Temperature over Time')
fig.show()

In [ ]:
fig = px.scatter(load_solar_df_inter.head(10000), x="datetime", y='Solar Zenith Angle', color='interpolated', title='Solar Zenith Angle over Time')
fig.show()

## Weather Data

In [13]:
DATA_DIR_WEATHER = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Weather/JFK"
weather_cols =['HourlyDryBulbTemperature','HourlyRelativeHumidity']



In [14]:
weeather_data = pd.read_csv(f"{DATA_DIR_WEATHER}/JFK_2017.csv",header=0)
for y in range(2018,2023):
  df = pd.read_csv(f"{DATA_DIR_WEATHER}/JFK_{y}.csv",header=0)
  weeather_data = weeather_data.append(df)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (64,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9,10,41,64,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9,10,23,64,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9,10,17,41,62,64) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-package

In [62]:
weeather_data

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,SOURCE,HourlyAltimeterSetting,HourlyDewPointTemperature,...,BackupDistanceUnit,BackupElements,BackupElevation,BackupEquipment,BackupLatitude,BackupLongitude,BackupName,WindEquipmentChangeDate,DYTS,DYHF
0,74486094789,2017-01-01T00:51:00,40.63915,-73.76401,3.4,"JFK INTERNATIONAL AIRPORT, NY US",FM-15,7,29.92,27,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
1,74486094789,2017-01-01T01:00:00,40.63915,-73.76401,3.4,"JFK INTERNATIONAL AIRPORT, NY US",FM-12,4,NaN,27,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
2,74486094789,2017-01-01T01:51:00,40.63915,-73.76401,3.4,"JFK INTERNATIONAL AIRPORT, NY US",FM-15,7,29.92,27,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
3,74486094789,2017-01-01T02:51:00,40.63915,-73.76401,3.4,"JFK INTERNATIONAL AIRPORT, NY US",FM-15,7,29.94,28,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
4,74486094789,2017-01-01T03:51:00,40.63915,-73.76401,3.4,"JFK INTERNATIONAL AIRPORT, NY US",FM-15,7,29.95,28,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13341,74486094789,2022-12-31T23:18:00,40.63915,-73.76390,2.7,"JFK INTERNATIONAL AIRPORT, NY US",FM-16,7,29.79,49,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
13342,74486094789,2022-12-31T23:49:00,40.63915,-73.76390,2.7,"JFK INTERNATIONAL AIRPORT, NY US",FM-16,6,29.77,48,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
13343,74486094789,2022-12-31T23:51:00,40.63915,-73.76390,2.7,"JFK INTERNATIONAL AIRPORT, NY US",FM-15,7,29.76,49,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN
13344,74486094789,2022-12-31T23:59:00,40.63915,-73.76390,2.7,"JFK INTERNATIONAL AIRPORT, NY US",SOD,6,NaN,NaN,...,mi,"TEMP, PRECIP, SNOW",NaN,"PSY, SRG, SNOWBOARD",NaN,NaN,FAA CWO,2009-05-01,NaN,NaN


In [15]:
weeather_data = weeather_data.drop_duplicates(subset='DATE')
weeather_data.index = pd.to_datetime(weeather_data['DATE'])
weeather_data = weeather_data.asfreq('1T')
# weeather_data[weather_cols]=weeather_data[weather_cols].astype(str).str.replace('s','')
weeather_data['HourlyDryBulbTemperature'] = weeather_data['HourlyDryBulbTemperature'].astype(str).str.replace('s','').astype(float)
weeather_data_inter = weeather_data[weather_cols].interpolate(method='linear', order=1, limit=200, limit_direction='both')
weeather_data_inter_5m = weeather_data_inter[weeather_data_inter.index.minute % 5 == 0]

In [11]:
weeather_data_inter_5m

,HourlyDryBulbTemperature,HourlyRelativeHumidity
DATE,,
2017-01-01 00:55:00,44.000000,51.000000
2017-01-01 01:00:00,44.000000,51.000000
2017-01-01 01:05:00,44.098039,50.803922
2017-01-01 01:10:00,44.196078,50.607843
2017-01-01 01:15:00,44.294118,50.411765
...,...,...
2022-12-31 23:35:00,48.451613,100.000000
2022-12-31 23:40:00,48.290323,100.000000
2022-12-31 23:45:00,48.129032,100.000000


In [26]:
merged  = load_solar_df.merge(weeather_data_inter_5m, how='left', right_index = True, left_index = True)

In [66]:
load_solar_df

,Time Stamp,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,...,GtiTracking,PrecipitableWater,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,dow_y
0,2017-01-01 00:00:00,1211.6000,1671.9000,620.2000,978.6000,1012.3000,2161.0000,867.2000,320.6000,5116.9000,...,0.0,14.5,43.5,0.0,1011.9,226.0,7.0,162.0,0.12,6.0
1,2017-01-01 00:05:00,1217.2000,1667.5000,610.4000,967.8000,1008.4000,2139.6000,848.1000,309.0000,5102.1000,...,0.0,14.5,43.5,0.0,1011.9,227.0,7.0,162.0,0.13,6.0
2,2017-01-01 00:10:00,1210.1000,1613.6000,622.1000,960.6000,947.0000,2131.8000,857.2000,326.9000,5074.9000,...,0.0,14.5,43.5,0.0,1011.9,227.0,7.0,162.0,0.13,6.0
3,2017-01-01 00:15:00,1205.2000,1629.4000,594.4000,953.4000,984.6000,2115.6000,866.2000,305.8000,5089.6000,...,0.0,14.5,43.5,0.0,1011.9,228.0,7.0,162.0,0.13,6.0
4,2017-01-01 00:20:00,1196.2000,1639.3000,599.2000,955.5000,991.1000,2110.6000,848.5000,305.3000,5061.3000,...,0.0,14.5,43.5,0.0,1011.9,228.0,7.0,162.0,0.13,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631075,2022-12-31 23:35:00,1173.4756,1455.1769,561.4635,938.2208,1000.8982,1920.7468,697.0698,238.4149,4749.6870,...,0.0,27.5,99.2,0.0,1014.8,252.0,2.5,162.0,0.10,5.0
631076,2022-12-31 23:40:00,1164.8765,1474.7970,563.8666,930.3062,946.4758,1906.6010,696.8730,236.1247,4738.3370,...,0.0,27.2,99.2,0.0,1014.7,251.0,2.4,162.0,0.10,5.0
631077,2022-12-31 23:45:00,1163.8800,1476.9541,561.3690,929.1260,952.0700,1891.1267,698.8709,229.8545,4731.2676,...,0.0,26.9,99.2,0.0,1014.6,250.0,2.4,162.0,0.10,5.0
631078,2022-12-31 23:50:00,1154.3290,1421.4199,561.5217,929.3459,928.7479,1879.3613,703.1354,234.4483,4721.4570,...,0.0,26.6,99.2,0.0,1014.5,249.0,2.3,162.0,0.10,5.0


In [ ]:
weeather_data["HourlyDryBulbTemperature"].value_counts()

73.0    2075
72.0    1973
75.0    1811
45.0    1729
74.0    1673
        ... 
96.0       5
4.0        4
3.0        4
98.0       3
97.0       2
Name: HourlyDryBulbTemperature, Length: 96, dtype: int64

In [ ]:
weeather_data_inter['HourlyDryBulbTemperature'].unique()

array([44.        , 44.01960784, 44.03921569, ..., 48.94339623,
       48.96226415, 48.98113208])

In [37]:
weeather_data_inter.interpolate(method='linear', order=1, limit=60, limit_direction='both')

,HourlyDryBulbTemperature,HourlyRelativeHumidity
DATE,,
2017-01-01 00:51:00,44.0,51.0
2017-01-01 00:52:00,44.0,51.0
2017-01-01 00:53:00,44.0,51.0
2017-01-01 00:54:00,44.0,51.0
2017-01-01 00:55:00,44.0,51.0
...,...,...
2022-12-31 23:55:00,49.0,100.0
2022-12-31 23:56:00,49.0,100.0
2022-12-31 23:57:00,49.0,100.0


In [ ]:
weeather_data_inter_5m.index

In [ ]:
print(load_solar_df_inter.shape, weeather_data_inter_5m.shape)

In [76]:
load_solar_df = load_solar_df.set_index('Time Stamp')

In [80]:
weeather_data_inter_5m

,DATE,HourlyDryBulbTemperature,HourlyRelativeHumidity,year,month,day,hour,minute
0,2017-01-01 00:55:00,44.000000,51.000000,2017,1,1,0,55
1,2017-01-01 01:00:00,44.000000,51.000000,2017,1,1,1,0
2,2017-01-01 01:05:00,44.098039,50.803922,2017,1,1,1,5
3,2017-01-01 01:10:00,44.196078,50.607843,2017,1,1,1,10
4,2017-01-01 01:15:00,44.294118,50.411765,2017,1,1,1,15
...,...,...,...,...,...,...,...,...
630992,2022-12-31 23:35:00,48.451613,100.000000,2022,12,31,23,35
630993,2022-12-31 23:40:00,48.290323,100.000000,2022,12,31,23,40
630994,2022-12-31 23:45:00,48.129032,100.000000,2022,12,31,23,45
630995,2022-12-31 23:50:00,48.500000,100.000000,2022,12,31,23,50


In [17]:
weeather_data_inter_5m = weeather_data_inter_5m.reset_index()
weeather_data_inter_5m['year'] = weeather_data_inter_5m['DATE'].apply(lambda x: x.year)
weeather_data_inter_5m['month'] = weeather_data_inter_5m['DATE'].apply(lambda x: x.month)
weeather_data_inter_5m['day'] = weeather_data_inter_5m['DATE'].apply(lambda x: x.day)
weeather_data_inter_5m['hour'] = weeather_data_inter_5m['DATE'].apply(lambda x: x.hour)
weeather_data_inter_5m['minute'] = weeather_data_inter_5m['DATE'].apply(lambda x: x.minute)
weeather_data_inter_5m = weeather_data_inter_5m.drop([ 'DATE'], axis = 1)

In [21]:
# merged = load_solar_df.merge(weeather_data_inter_5m, right_index=True, left_index=True, how='left')

merged = load_solar_df.merge(weeather_data_inter_5m, how='left', left_on=['year','month',	'day',	'hour',	'minute'], right_on=['year','month',	'day',	'hour',	'minute'])

In [22]:
merged

,Time Stamp,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,...,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,dow_y,HourlyDryBulbTemperature,HourlyRelativeHumidity
0,2017-01-01 00:00:00,1211.6000,1671.9000,620.2000,978.6000,1012.3000,2161.0000,867.2000,320.6000,5116.9000,...,43.5,0.0,1011.9,226.0,7.0,162.0,0.12,6.0,NaN,NaN
1,2017-01-01 00:05:00,1217.2000,1667.5000,610.4000,967.8000,1008.4000,2139.6000,848.1000,309.0000,5102.1000,...,43.5,0.0,1011.9,227.0,7.0,162.0,0.13,6.0,NaN,NaN
2,2017-01-01 00:10:00,1210.1000,1613.6000,622.1000,960.6000,947.0000,2131.8000,857.2000,326.9000,5074.9000,...,43.5,0.0,1011.9,227.0,7.0,162.0,0.13,6.0,NaN,NaN
3,2017-01-01 00:15:00,1205.2000,1629.4000,594.4000,953.4000,984.6000,2115.6000,866.2000,305.8000,5089.6000,...,43.5,0.0,1011.9,228.0,7.0,162.0,0.13,6.0,NaN,NaN
4,2017-01-01 00:20:00,1196.2000,1639.3000,599.2000,955.5000,991.1000,2110.6000,848.5000,305.3000,5061.3000,...,43.5,0.0,1011.9,228.0,7.0,162.0,0.13,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631003,2022-12-31 23:35:00,1173.4756,1455.1769,561.4635,938.2208,1000.8982,1920.7468,697.0698,238.4149,4749.6870,...,99.2,0.0,1014.8,252.0,2.5,162.0,0.10,5.0,48.451613,100.0
631004,2022-12-31 23:40:00,1164.8765,1474.7970,563.8666,930.3062,946.4758,1906.6010,696.8730,236.1247,4738.3370,...,99.2,0.0,1014.7,251.0,2.4,162.0,0.10,5.0,48.290323,100.0
631005,2022-12-31 23:45:00,1163.8800,1476.9541,561.3690,929.1260,952.0700,1891.1267,698.8709,229.8545,4731.2676,...,99.2,0.0,1014.6,250.0,2.4,162.0,0.10,5.0,48.129032,100.0
631006,2022-12-31 23:50:00,1154.3290,1421.4199,561.5217,929.3459,928.7479,1879.3613,703.1354,234.4483,4721.4570,...,99.2,0.0,1014.5,249.0,2.3,162.0,0.10,5.0,48.500000,100.0


In [23]:
merged= merged.bfill().ffill()

In [ ]:
merged1[merged1["year"].isin([2017,2018,2019,2020,2021,2022])]['HourlyDryBulbTemperature'].value_counts()

73.000000    6183
72.000000    6164
45.000000    5192
75.000000    5100
42.000000    4805
             ... 
7.431373        1
7.235294        1
7.039216        1
4.933333        1
48.981132       1
Name: HourlyDryBulbTemperature, Length: 25533, dtype: int64

In [ ]:
merged1[merged1["year"].isin([2017,2018,2019,2020,2021,2022])]['HourlyRelativeHumidity'].value_counts()

100.000000    40816
93.000000     10733
96.000000      6596
90.000000      5571
97.000000      5372
              ...  
33.843137         1
92.172414         1
91.482759         1
89.744681         1
58.428571         1
Name: HourlyRelativeHumidity, Length: 22852, dtype: int64

In [ ]:
merged_2017_2022 = merged1[merged1["year"].isin([2017,2018,2019,2020,2021,2022])]

In [ ]:
merged_2017_2022.isna().sum(axis = 0)

Time Stamp                  0
CAPITL                      0
CENTRL                      0
DUNWOD                      0
GENESE                      0
HUD VL                      0
LONGIL                      0
MHK VL                      0
MILLWD                      0
N.Y.C.                      0
NORTH                       0
WEST                        0
NYCA                        0
datetime_x                  0
year                        0
month                       0
day                         0
hour                        0
minute                      0
dow_x                       0
PeriodEnd                   0
PeriodStart                 0
Period                      0
AirTemp                     0
Azimuth                     0
CloudOpacity                0
DewpointTemp                0
Dhi                         0
Dni                         0
Ebh                         0
Ghi                         0
GtiFixedTilt                0
GtiTracking                 0
Precipitab

In [ ]:
merged_2017_2022['HourlyRelativeHumidity'].value_counts()

100.000000    40816
93.000000     10733
96.000000      6596
90.000000      5571
97.000000      5372
              ...  
33.843137         1
92.172414         1
91.482759         1
89.744681         1
58.428571         1
Name: HourlyRelativeHumidity, Length: 22852, dtype: int64

In [ ]:
for temp in merged_2017_2022['HourlyDryBulbTemperature'].unique():
  print(temp)

Streaming output truncated to the last 5000 lines.
73.42424242424242
73.57575757575758
73.72727272727273
73.87878787878788
73.95652173913044
73.52173913043478
73.30434782608695
70.80392156862746
63.90909090909091
63.79545454545455
63.68181818181818
63.56818181818182
63.45454545454545
63.34090909090909
63.22727272727273
63.11363636363637
63.391304347826086
63.82608695652174
64.26086956521739
64.69565217391305
68.19607843137254
72.9795918367347
72.16326530612245
74.82758620689656
74.48275862068965
74.13793103448276
70.93617021276596
70.82978723404256
70.72340425531915
70.61702127659575
70.51063829787235
70.40425531914893
70.29787234042553
70.19148936170212
70.08510638297872
69.13888888888889
69.27777777777777
69.69444444444444
69.97222222222223
70.225
70.475
70.725
70.975
74.76190476190476
74.52380952380952
74.04761904761905
74.10810810810811
74.24324324324324
74.37837837837837
74.51351351351352
74.64864864864865
74.78378378378379
74.91891891891892
74.91304347826087
74.69565217391305
74.

In [ ]:
fig = px.scatter(merged_2017_2022.head(100000), x="datetime_y", y='HourlyDryBulbTemperature', title='Temperature over Time')
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = px.scatter(merged_2017_2022.head(100000), x="datetime_y", y='HourlyDryBulbTemperature', title='Temperature over Time')
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = px.scatter(merged_2017_2022.head(100000), x="datetime_y", y='HourlyRelativeHumidity', title='Temperature over Time')
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = px.scatter(merged_2017_2022.head(100000), x="datetime_y", y='HourlyRelativeHumidity', title='Temperature over Time')
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [24]:
merged.to_csv(f"{SOLAR_DATA_DIR}/full_merge_v3.csv")

In [84]:
merged

,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,NORTH,...,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,dow_y,DATE,HourlyDryBulbTemperature,HourlyRelativeHumidity
0,1211.6000,1671.9000,620.2000,978.6000,1012.3000,2161.0000,867.2000,320.6000,5116.9000,539.2000,...,0.0,1011.9,226.0,7.0,162.0,0.12,6.0,NaT,NaN,NaN
1,1217.2000,1667.5000,610.4000,967.8000,1008.4000,2139.6000,848.1000,309.0000,5102.1000,542.5000,...,0.0,1011.9,227.0,7.0,162.0,0.13,6.0,NaT,NaN,NaN
2,1210.1000,1613.6000,622.1000,960.6000,947.0000,2131.8000,857.2000,326.9000,5074.9000,527.8000,...,0.0,1011.9,227.0,7.0,162.0,0.13,6.0,NaT,NaN,NaN
3,1205.2000,1629.4000,594.4000,953.4000,984.6000,2115.6000,866.2000,305.8000,5089.6000,531.5000,...,0.0,1011.9,228.0,7.0,162.0,0.13,6.0,NaT,NaN,NaN
4,1196.2000,1639.3000,599.2000,955.5000,991.1000,2110.6000,848.5000,305.3000,5061.3000,538.2000,...,0.0,1011.9,228.0,7.0,162.0,0.13,6.0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631003,1173.4756,1455.1769,561.4635,938.2208,1000.8982,1920.7468,697.0698,238.4149,4749.6870,661.1154,...,0.0,1014.8,252.0,2.5,162.0,0.10,5.0,2022-12-31 23:35:00,48.451613,100.0
631004,1164.8765,1474.7970,563.8666,930.3062,946.4758,1906.6010,696.8730,236.1247,4738.3370,657.3319,...,0.0,1014.7,251.0,2.4,162.0,0.10,5.0,2022-12-31 23:40:00,48.290323,100.0
631005,1163.8800,1476.9541,561.3690,929.1260,952.0700,1891.1267,698.8709,229.8545,4731.2676,658.4504,...,0.0,1014.6,250.0,2.4,162.0,0.10,5.0,2022-12-31 23:45:00,48.129032,100.0
631006,1154.3290,1421.4199,561.5217,929.3459,928.7479,1879.3613,703.1354,234.4483,4721.4570,653.5872,...,0.0,1014.5,249.0,2.3,162.0,0.10,5.0,2022-12-31 23:50:00,48.500000,100.0


In [ ]:
merged_2017_2022.to_csv(f"{SOLAR_DATA_DIR}/full_merge_v2.csv")

In [29]:
SOLAR_DATA_DIR

'/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data/Solar_Irradiance'

In [ ]:
merged.shape

In [ ]:
fig = px.scatter(merged, x='Temperature', y='HourlyDryBulbTemperature', title='Interpolated Load vs Temperature')
fig.update_layout(
    autosize=False,
    width=1000,
    height=800
)
fig.show()